# 교차 검증
- cross_val_score()
- kfold를 단순화 시킨다고 생각하면 됨

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np 

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, 
                         scoring = 'accuracy', cv=3)

# cross_val_score() 에서 회귀일때는 KFold(회귀에서는 StratifiedKFold를 적용할 수가 없다)
# 분류일때는 자동으로 StratifiedKFold로 자동적으로 적용이 됨

- GridSearchCV
- for 구문으로 돌리는 것보다 더 이득임

In [7]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data,
                                                   iris.target,
                                                   test_size = 0.3,
                                                   random_state=121)

dtree = DecisionTreeClassifier()

parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [10]:
import pandas as pd 

grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, 
                          refit=True, return_train_score=True)

# 저절로 GridSearchCV를 통해 하이퍼 파라미터들을 순차적으로 학습 / 평가 함
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과는  cv_results_라는 딕셔너리로 저장됨.
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score','rank_test_score',
          'split0_test_score', 'split1_test_score', 'split2_test_score']]


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.657143,5,0.657143,0.657143,0.657143
1,"{'max_depth': 1, 'min_samples_split': 3}",0.657143,5,0.657143,0.657143,0.657143
2,"{'max_depth': 2, 'min_samples_split': 2}",0.933333,3,0.942857,0.914286,0.942857
3,"{'max_depth': 2, 'min_samples_split': 3}",0.933333,3,0.942857,0.914286,0.942857
4,"{'max_depth': 3, 'min_samples_split': 2}",0.942857,1,0.971429,0.914286,0.942857
5,"{'max_depth': 3, 'min_samples_split': 3}",0.942857,1,0.971429,0.914286,0.942857


In [14]:
# grid_dtree.cv_results_

In [13]:
print('최적 파라미터', grid_dtree.best_params_)
print('최고 정확도', grid_dtree.best_score_)

# refit = True 로 설정된 GridSearchCV 객체가 fit()을 수행 시 학습이 완료된
# Estimator를 내포하고 있으므로 predict()를 통해 예측도 가능하다.
# refit = True를 하면 다시 grid_dtree.fit(x_train, y_train)를 
# 다시 할 필요가 없는것이다

pred = grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'
      .format(accuracy_score(y_test,pred)))

최적 파라미터 {'max_depth': 3, 'min_samples_split': 2}
최고 정확도 0.9428571428571427
테스트 데이터 세트 정확도: 0.9556


In [16]:
# GridSearchCV 의 refit으로 인해 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_parametor_는 이미 최적 하이퍼 파라미터로 학습이 되어 있음.
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도 : {0:.4f}'.format(accuracy_score(y_test, pred)))


테스트 데이터 세트 정확도 : 0.9556
